In [ ]:
! pip install SimpleITK
! pip install nibabel

In [83]:
import SimpleITK as sitk
import numpy as np
from scipy import ndimage
import math
import cv2
import random
import sys
import math
import requests
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import nibabel as nib
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from torchvision import datasets, models, transforms
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression


In [82]:
%run "C:\Users\johnm\Downloads\utils.ipynb"  #tarang

# Merging images with label in one dictionary

In [86]:
# tarang updated this to make it a function

def make_dict(directory, multi_fold = False, dict_images = {}):

  if multi_fold:
    print('fold')
    folds = os.listdir(directory)
    for f in folds:
      fold_path = os.path.join(directory, f)
      print(fold_path)
      make_dict(fold_path, False, dict_images)
    return dict_images
  else:
    print('datapoint')
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder) # /Users/kimberlyllajarunaperalta/Documents/ComputerVision/project/picai_public_images_fold1/10010/10010_1000010
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if "t2w" in file_name and file_name.endswith('.mha'):
                    full_path = os.path.join(directory, folder, file_name)
                    if os.path.exists(full_path):
                        short_name = '_'.join(file_name.split('_')[:2])
                        image = sitk.ReadImage(full_path)
                        dict_images[short_name] = {'image': image, 'case_pca': None, 'patient_age': None, 'age_group': None, 'patient_id': None}
    return dict_images


In [87]:
dict_images = {}

directory = "D:\Downloads\Luna Training Data"
folds = os.listdir(directory)

# print(folds)

dict_images = make_dict(directory, True, dict_images)

<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\johnm\AppData\Local\Temp\ipykernel_14600\1865082310.py:3: SyntaxWarning: invalid escape sequence '\D'
  directory = "D:\Downloads\Luna Training Data"


fold
D:\Downloads\Luna Training Data\picai_public_images_fold0
datapoint
D:\Downloads\Luna Training Data\picai_public_images_fold1
datapoint
D:\Downloads\Luna Training Data\picai_public_images_fold2
datapoint
D:\Downloads\Luna Training Data\picai_public_images_fold3
datapoint
D:\Downloads\Luna Training Data\picai_public_images_fold4
datapoint


In [7]:
print(list(dict_images.keys())[0])

dict_images[list(dict_images.keys())[0]]

10000_1000000


{'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x0000024200B13D80> >,
 'case_pca': None,
 'patient_age': None,
 'age_group': None,
 'patient_id': None}

In [88]:
#size of the dictionary
len(dict_images)

1500

We stored 3 things:
* the nii files that have the color coded images
* the labels for each image (from csv file)
* the age of patients for each image (from csv file)

# Using the CSV file to get the labels and patients' age

In this spreadsheet, there is a column "lesion_GS", which lists the outcomes:

Gleason score (GS) assigned to each lesion after histopathologic analysis, where scores for different lesions are separated by , (commas). If its value is missing, then no tissue sampling procedure was performed; indicating a negative MRI study. If its value is N/A only for specific lesion(s), then those lesion(s) (as observed in radiology) were not biopsied or graded in histopathology (typically the case for PI-RADS 1-2 lesions).

This means that any case that contains a Gleason score of 7 (3+4 or higher) harbors csPCa, while all other cases are considered negative for csPCa. For more details on how this ground truth was derived, you can also check out our study design.


In [89]:
# tarang

labels_path = "C:\\Users\johnm\Downloads\marksheet.csv"

<>:3: SyntaxWarning: invalid escape sequence '\j'
<>:3: SyntaxWarning: invalid escape sequence '\j'
C:\Users\johnm\AppData\Local\Temp\ipykernel_14600\299217126.py:3: SyntaxWarning: invalid escape sequence '\j'
  labels_path = "C:\\Users\johnm\Downloads\marksheet.csv"


In [90]:
df = pd.read_csv(labels_path)

In [11]:
df.head(100)

,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,lesion_ISUP,case_ISUP,case_csPCa
0,10000,1000000,2019-07-02,73,7.70,NaN,55.0,MRBx,0+0,0,0,NO
1,10001,1000001,2016-05-27,64,8.70,0.09,102.0,NaN,NaN,NaN,0,NO
2,10002,1000002,2021-04-18,58,4.20,0.06,74.0,NaN,NaN,NaN,0,NO
3,10003,1000003,2019-04-05,72,13.00,NaN,71.5,SysBx,0+0,0,0,NO
4,10004,1000004,2020-10-21,67,8.00,0.10,78.0,SysBx+MRBx,"0+0,0+0","0,0",0,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
95,10095,1000095,2017-02-09,65,9.20,NaN,60.0,MRBx,0+0,0,0,NO
96,10096,1000096,2011-12-31,64,22.33,0.66,34.0,MRBx,0+0,0,0,NO
97,10097,1000097,2018-11-27,66,15.01,NaN,32.0,MRBx,3+4,2,2,YES
98,10098,1000098,2012-03-29,69,12.00,0.17,69.0,NaN,NaN,NaN,0,NO


### Evaluating class imbalance

In [12]:
df['case_csPCa'].value_counts()

case_csPCa
NO     1075
YES     425
Name: count, dtype: int64

Combining them both

## Combining patient_id and study_id to get the full differentiator per patient

In [91]:
df['full_name'] = df['patient_id'].astype(str) + '_' + df['study_id'].astype(str)

In [14]:
df

,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,lesion_ISUP,case_ISUP,case_csPCa,full_name
0,10000,1000000,2019-07-02,73,7.70,NaN,55.0,MRBx,0+0,0,0,NO,10000_1000000
1,10001,1000001,2016-05-27,64,8.70,0.09,102.0,NaN,NaN,NaN,0,NO,10001_1000001
2,10002,1000002,2021-04-18,58,4.20,0.06,74.0,NaN,NaN,NaN,0,NO,10002_1000002
3,10003,1000003,2019-04-05,72,13.00,NaN,71.5,SysBx,0+0,0,0,NO,10003_1000003
4,10004,1000004,2020-10-21,67,8.00,0.10,78.0,SysBx+MRBx,"0+0,0+0","0,0",0,NO,10004_1000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,11471,1001495,2012-08-25,71,12.50,0.21,62.0,MRBx,"3+4,N/A,3+3","2,1",2,YES,11471_1001495
1496,11472,1001496,2019-06-28,81,5.28,0.12,44.0,SysBx+MRBx,3+4,2,2,YES,11472_1001496
1497,11473,1001497,2017-09-24,56,29.60,0.34,87.0,MRBx,0+0,0,0,NO,11473_1001497
1498,11474,1001498,2016-05-03,71,12.00,NaN,83.0,MRBx,3+3,1,1,NO,11474_1001498


### Testing

In [92]:
specific_full_name = '10000_1000000'
case_csPCA_value = df.loc[df['full_name'] == specific_full_name, 'case_csPCa'].iloc[0]
case_csPCA_value

'NO'

## Adding to a dictionary based on their full name

In [94]:
not_found = 0
found = 0

for f in df['full_name']:
    if f in dict_images:
        print(f"Found {f}")
        case_csPCa_value = df.loc[df['full_name'] == f, 'case_csPCa'].iloc[0]
        patient_age = df.loc[df['full_name'] == f, 'patient_age'].iloc[0]
        #converting to 0 and 1
        dict_images[f]['case_pca'] = 0 if case_csPCa_value == 'NO' else 1 if case_csPCa_value == 'YES' else None
        dict_images[f]['patient_age'] = patient_age
        dict_images[f]['patient_id'] = f

        age_group = categorize_age(patient_age)
        dict_images[f]['age_group'] = age_group

        found += 1
    else:
        print(f"Could not find {f}")
        not_found += 1

Found 10000_1000000
Found 10001_1000001
Found 10002_1000002
Found 10003_1000003
Found 10004_1000004
Found 10005_1000005
Found 10006_1000006
Found 10007_1000007
Found 10008_1000008
Found 10009_1000009
Found 10010_1000010
Found 10011_1000011
Found 10012_1000012
Found 10013_1000013
Found 10014_1000014
Found 10015_1000015
Found 10016_1000016
Found 10017_1000017
Found 10018_1000018
Found 10019_1000019
Found 10020_1000020
Found 10021_1000021
Found 10022_1000022
Found 10023_1000023
Found 10024_1000024
Found 10025_1000025
Found 10026_1000026
Found 10027_1000027
Found 10028_1000028
Found 10029_1000029
Found 10030_1000030
Found 10031_1000031
Found 10032_1000032
Found 10033_1000033
Found 10034_1000034
Found 10035_1000035
Found 10036_1000036
Found 10037_1000037
Found 10038_1000038
Found 10039_1000039
Found 10040_1000040
Found 10041_1000041
Found 10042_1000042
Found 10043_1000043
Found 10044_1000044
Found 10045_1000045
Found 10046_1000046
Found 10047_1000047
Found 10048_1000048
Found 10049_1000049


In [95]:
found, not_found

(1500, 0)

In [96]:
dict_images

{'10000_1000000': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246FDFCBDE0> >,
  'case_pca': 0,
  'patient_age': 73,
  'age_group': '65+',
  'patient_id': '10000_1000000'},
 '10001_1000001': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246EAEF3090> >,
  'case_pca': 0,
  'patient_age': 64,
  'age_group': '<65',
  'patient_id': '10001_1000001'},
 '10003_1000003': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246EAEF17D0> >,
  'case_pca': 0,
  'patient_age': 72,
  'age_group': '65+',
  'patient_id': '10003_1000003'},
 '10006_1000006': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246F1D06C10> >,
  'case_pca': 0,
  'patient_age': 73,
  'age_group': '65+',
  'patient_id': '10006_1000006'},
 '10017_1000017': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of typ

### Evaluating class imbalance in final sample

In [97]:
def balance_checker(current_dictionary, return_counts = False):
    # Extract 'case_pca' values from dictionary values
    case_pca_values = [item['case_pca'] for item in current_dictionary.values()]

    # Count occurrences of 'YES' and 'NO'
    counts = Counter(case_pca_values)
    yes_count = counts[1]
    no_count = counts[0]

    if return_counts:
        return yes_count, no_count
    else:
        print(f"Count of 'YES': {yes_count}")
        print(f"Count of 'NO': {no_count}")


In [98]:
balance_checker(dict_images)

Count of 'YES': 425
Count of 'NO': 1075


In [99]:
#our dictionary with the T2w images and the case_csPCa values
dict_images

{'10000_1000000': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246FDFCBDE0> >,
  'case_pca': 0,
  'patient_age': 73,
  'age_group': '65+',
  'patient_id': '10000_1000000'},
 '10001_1000001': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246EAEF3090> >,
  'case_pca': 0,
  'patient_age': 64,
  'age_group': '<65',
  'patient_id': '10001_1000001'},
 '10003_1000003': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246EAEF17D0> >,
  'case_pca': 0,
  'patient_age': 72,
  'age_group': '65+',
  'patient_id': '10003_1000003'},
 '10006_1000006': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246F1D06C10> >,
  'case_pca': 0,
  'patient_age': 73,
  'age_group': '65+',
  'patient_id': '10006_1000006'},
 '10017_1000017': {'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of typ

In [100]:
dict_images[list(dict_images.keys())[0]]

{'image': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x00000246FDFCBDE0> >,
 'case_pca': 0,
 'patient_age': 73,
 'age_group': '65+',
 'patient_id': '10000_1000000'}

# Image Processing and Train Test Split

In [101]:
def preprocess_data(data):
    images, labels, ages = [], [], []

    for entry in data.values():
        image = forresnet(entry['image'])
        label = entry['case_pca']
        age = entry['patient_age']
        images.append(image)
        labels.append(label)
        ages.append(age)

    X_train, X_test, y_train, y_test, age_train, age_test = train_test_split(images, labels, ages, test_size = 0.2, random_state = 42)
    X_train, X_val, y_train, y_val, age_train, age_val = train_test_split(X_train, y_train, age_train, test_size = 0.2, random_state = 42)

    X_train = np.array(X_train)
    X_val = np.array(X_val)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_val = np.array(y_val)
    y_test = np.array(y_test)
    age_train = np.array(age_train)
    age_val = np.array(age_val)
    age_test = np.array(age_test)

    return X_train, X_val, X_test, y_train, y_val, y_test, age_train, age_val, age_test

In [102]:
X_train, X_val, X_test, y_train, y_val, y_test, age_train, age_val, age_test = preprocess_data(dict_images)

Streaming output truncated to the last 5000 lines.
Shape after stacking: (224, 224, 3)
Shape after normalization: (224, 224, 3)
Original shape: (384, 384)
Shape after adding dimension: (384, 384, 1)
Resized shape: (224, 224, 1)
Shape after squeezing: (224, 224)
Shape after stacking: (224, 224, 3)
Shape after normalization: (224, 224, 3)
Original shape: (384, 384)
Shape after adding dimension: (384, 384, 1)
Resized shape: (224, 224, 1)
Shape after squeezing: (224, 224)
Shape after stacking: (224, 224, 3)
Shape after normalization: (224, 224, 3)
Original shape: (384, 384)
Shape after adding dimension: (384, 384, 1)
Resized shape: (224, 224, 1)
Shape after squeezing: (224, 224)
Shape after stacking: (224, 224, 3)
Shape after normalization: (224, 224, 3)
Original shape: (640, 640)
Shape after adding dimension: (640, 640, 1)
Resized shape: (224, 224, 1)
Shape after squeezing: (224, 224)
Shape after stacking: (224, 224, 3)
Shape after normalization: (224, 224, 3)
Original shape: (640, 640)
S

# Evaluating class imbalance in train, test, and validation sets

In [26]:
# Count labels in X_train
train_label_counts = Counter(y_train)
print("Train Label Counts:", train_label_counts)

# Count labels in X_val
val_label_counts = Counter(y_val)
print("Validation Label Counts:", val_label_counts)

# Count labels in X_test
test_label_counts = Counter(y_test)
print("Test Label Counts:", test_label_counts)


Train Label Counts: Counter({0: 701, 1: 259})
Validation Label Counts: Counter({0: 166, 1: 74})
Test Label Counts: Counter({0: 208, 1: 92})


In [27]:
X_train.shape, X_val.shape, X_test.shape

((960, 224, 224, 3), (240, 224, 224, 3), (300, 224, 224, 3))

## Addressing Class Imbalance in Train set
Done after train-test split to avoid data leakage.

In [28]:
def resample_train_data(X_train, y_train, age_train, upsampling = True, random_state = 42):

    #getting majority and minority classes
    unique_classes, class_counts = np.unique(y_train, return_counts=True)
    majority_label = unique_classes[np.argmax(class_counts)]
    minority_label = unique_classes[np.argmin(class_counts)]

    majority_indices = np.where(y_train == majority_label)[0]
    minority_indices = np.where(y_train == minority_label)[0]
    # print(majority_indices)
    # print(minority_indices)

    if upsampling:
        minority_upsampled_indices = resample(minority_indices, replace = True, n_samples = len(majority_indices), random_state = random_state)
        resampled_indices = np.concatenate((majority_indices, minority_upsampled_indices))
    else:
        majority_downsampled_indices = resample(majority_indices, replace = False, n_samples = len(minority_indices), random_state = random_state)
        resampled_indices = np.concatenate((majority_downsampled_indices, minority_indices))

    #selecting the samples corresponding to the resampled indices
    X_train_resampled = X_train[resampled_indices]
    y_train_resampled = y_train[resampled_indices]
    age_train_resampled = age_train[resampled_indices]

    return X_train_resampled, y_train_resampled, age_train_resampled


In [ ]:
def more_cancer_samples(X_train, y_train, age_train, proportion = 1.25, random_state = 42):
    cancer_indeces = np.where(y_train == 1)[0]
    no_cancer_indeces = np.where(y_train == 0)[0]

    desired_cancer_indeces = int(proportion * len(no_cancer_indeces))

    #upsampling the cancer class to match the given proportion
    cancer_upsampled_indices = resample(
        cancer_indeces, replace = True, n_samples = desired_cancer_indeces, random_state = random_state)

    #combining minority upsampled indices with majority indices
    adjusted_indices = np.concatenate((no_cancer_indeces, cancer_upsampled_indices))

    #selecting the corresponding samples from the original data
    X_adjusted = X_train[adjusted_indices]
    y_adjusted = y_train[adjusted_indices]
    age_adjusted = age_train[adjusted_indices]

    return X_adjusted, y_adjusted, age_adjusted

In [29]:
def list_balance_checker(X_train, y_train, age_train):
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)
    print("age_train shape:", age_train.shape)

    counts = np.bincount(y_train)
    print('Number of 0:', counts[0])
    print('Number of 1:', counts[1])

### Upsampling

In [30]:
### Before upsampling
list_balance_checker(X_train, y_train, age_train)

X_train shape: (960, 224, 224, 3)
y_train shape: (960,)
age_train shape: (960,)
Number of 0: 701
Number of 1: 259


In [31]:
### Upsampling
uX_train, uy_train, uage_train = resample_train_data(X_train, y_train, age_train, upsampling = True)

### Downsampling
dX_train, dy_train, dage_train = resample_train_data(X_train, y_train, age_train, upsampling = False)

### Upsamping > Cancer
mX_train, my_train, mage_train = resample_train_data(X_train, y_train, age_train, upsampling = True)


In [32]:
print("Upsampled:")
list_balance_checker(uX_train, uy_train, uage_train)

print("Downsampled")
list_balance_checker(dX_train, dy_train, dage_train)

print("More Cancer")
list_balance_checker(dX_train, dy_train, dage_train)

X_train shape: (1402, 224, 224, 3)
y_train shape: (1402,)
age_train shape: (1402,)
Number of 0: 701
Number of 1: 701


# Baseline Model: ResNet50

# Experiment 1: Hyperparameter Training

In [ ]:
# for upsampling, downsampling, and more cancer:

# learning rates: (0.01, 0.005, 0.001, 0.0001?)
# batch size (16, 32, 64)
# epochs (10, 40, 70)

In [34]:
# function form of model code above

def get_model(learning_rate, verbose = False):
  # Load pre-trained ResNet-50 model without top layers
  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

  # Freeze the base model layers
  for layer in base_model.layers:
    layer.trainable = False

  # Add new classification layers
  x = base_model.output
  x = GlobalAveragePooling2D()(x)  # Add a global average pooling layer
  x = Dense(128, activation='relu')(x)  # Add a dense layer
  predictions = Dense(1, activation='sigmoid')(x)  # Add an output layer for binary classification

  # Combine the base model and new layers
  model = Model(inputs=base_model.input, outputs=predictions)

  # Compile the model
  model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

  # Print model summary
  if verbose:
    model.summary()

  return model


In [104]:
def run_hypers(X_train, y_train):
  lrms = {}
  batchms = {}
  epochms = {}

  for lr in lrs:
    print("training with lr: ", lr)
    model = get_model(lr)
    early_stopping = EarlyStopping(monitor='val_loss', patience = 6, restore_best_weights = True)

    # Train the model
    history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks = [early_stopping])

    lrms[lr] = model

  for batch in batches:
    print("training with batch size: ", batch)
    model = get_model(0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience = 6, restore_best_weights = True)

    # Train the model
    history = model.fit(X_train, y_train,
                      epochs=10,
                      batch_size=batch,
                      validation_data=(X_val, y_val),
                      callbacks = [early_stopping])

    batchms[batch] = model

  for epoch in epochs:
    print("training with epochs: ", epoch)
    model = get_model(0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience = 6, restore_best_weights = True)

    # Train the model
    history = model.fit(X_train, y_train,
                      epochs=epoch,
                      batch_size=32,
                      validation_data=(X_val, y_val),
                      callbacks = [early_stopping])

    epochms[epoch] = model

  test_metrics = {}

  for lr in lrs:
    print("learning rate: ", lr)
    test_loss, test_acc = lrms[lr].evaluate(X_test, y_test)
    test_metrics["lr" + str(lr)] = {'loss' : test_loss,
                              'accuracy' : test_acc}

  for batch in batches:
    print("batch size: ", batch)
    test_loss, test_acc = batchms[batch].evaluate(X_test, y_test)
    test_metrics["batch" + str(batch)] = {'loss' : test_loss,
                              'accuracy' : test_acc}

  for epoch in epochs:
    print("epochs: ", epoch)
    test_loss, test_acc = epochms[epoch].evaluate(X_test, y_test)
    test_metrics["epoch" + str(epoch)] = {'loss' : test_loss,
                              'accuracy' : test_acc}

  return lrms, batchms, epochms, test_metrics

In [35]:
lrs = [0.01, 0.005, 0.001, 0.0001, 0.05]
batches = [16, 32, 64]
epochs = [10, 40, 70]

## Hyper-parameter Training

In [ ]:
uplrs, upbatches, upepochs, upmetrics = run_hypers(uX_train, uy_train)

print(upmetrics)

In [ ]:
downlrs, downbatches, downepochs, downmetrics = run_hypers(dX_train, dy_train)

print(downmetrics)

In [ ]:
morelrs, morebatches, moreepochs, moremetrics = run_hypers(mX_train, my_train)

print(moremetrics)

## Evaluating Best Models (TODO FROM HERE DOWN)

In [ ]:
evaluate_model(up_epochs[70], 0.5, X_train, y_train, X_val, y_val, X_test, y_test, include_test = True)

## Model with All Best Hyperparameters (Upsampling)

In [ ]:

model = get_model(0.005)

# ADD EARLY STOPPING

# Train the model
history = model.fit(X_train, y_train,
                    epochs=70,
                    batch_size=64,
                    validation_data=(X_val, y_val))

bestUpModel = model

bestUpModel

In [76]:
test_loss, test_acc = bestUpModel.evaluate(X_test, y_test)
bestUpTestMetrics = {'loss' : test_loss,
                     'accuracy' : test_acc}

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 673ms/step - accuracy: 0.6388 - loss: 1.2715


In [ ]:
evaluate_model(bestUpModel, 0.5, X_train, y_train, X_val, y_val, X_test, y_test, include_test = True)

## Downsampling Hyperparameter